<a href="https://colab.research.google.com/github/hoangducsn00/webScrape_tiki/blob/main/scrapingweb_tikicomment_1-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/My Drive/devC/clone_data
!ls

/content/drive/My Drive/devC/clone_data
 crawl-tiki-products  ' SCRAPE DYNAMIC websites.ipynb'
 data		       scrapingweb_tikicomment.ipynb
 data_comment_tiki


In [ ]:
#import library
from bs4 import BeautifulSoup
import requests
import json
import csv
import pandas as pd
import urllib, json

In [ ]:
names = ['ao+dai', 'giay+the+thao', 'giay+nam', 'giay+cao+got', 'sach+giao+khoa', 'truyện+tranh', 'ao+somi', 'do+dung+hoc+tap',
         'mu+bao+hiem', 'hoodie', 'ao+len', 'tat', 'tai+nghe+bluetooth', 'tinh+dầu', 'balo', 'quan+au', 'noi+com+dien', "kho+ga", "laptop"
         , 'tã+quần', 'bột+giặt', 'bàn+học', 'đèn+học', 'lăn+khử+mùi', 'nước+hoa']

In [ ]:

def crawl_product_id():
    product_list = []
    i = 1
    while (True):
        print("Crawl page: {} {}".format(i, name))
        response = requests.get(laptop_page_url.format(i))
        parser = BeautifulSoup(response.text, 'html.parser')

        product_box = parser.findAll(class_="product-item")

        if (len(product_box) == 0):
            break

        for product in product_box:
            product_list.append(product.get("data-id"))

        i += 1
        if (i == 2):
          break


    return product_list, i

def save_product_id(product_list=[]):
    file = open(product_id_file, "w+")
    str = "\n".join(product_list)
    file.write(str)
    file.close()
    print("Save file: ", product_id_file)

def crawl_product(product_list=[]):
    product_detail_list = []
    for product_id in product_list:
        response = requests.get(product_url.format(product_id))
        if (response.status_code == 200):
            product_detail_list.append(response.text)
            print("Crawl product: ", product_id, ": ", response.status_code)
    return product_detail_list



In [ ]:
for name in names:
  for i in range(1,4):
    laptop_page_url = "https://tiki.vn/search?q=" + name + '&order=top_seller&page=' + str(i)
    product_url = "https://tiki.vn/search?q=" + name
    product_id_file = "/content/drive/My Drive/clone_cmt_tiki/id_product/" + name + str(i) + "_id.txt"
    product_list, page = crawl_product_id()
    save_product_id(product_list)

Crawl page: 1 ao+dai
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/ao+dai1_id.txt
Crawl page: 1 ao+dai
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/ao+dai2_id.txt
Crawl page: 1 ao+dai
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/ao+dai3_id.txt
Crawl page: 1 giay+the+thao
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/giay+the+thao1_id.txt
Crawl page: 1 giay+the+thao
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/giay+the+thao2_id.txt
Crawl page: 1 giay+the+thao
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/giay+the+thao3_id.txt
Crawl page: 1 giay+nam
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/giay+nam1_id.txt
Crawl page: 1 giay+nam
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/giay+nam2_id.txt
Crawl page: 1 giay+nam
Save file:  /content/drive/My Drive/clone_cmt_tiki/id_product/giay+nam3_id.txt
Crawl page: 1 giay+cao+got
Save file:  /content/drive/My Drive/c

In [ ]:
list_data = []
def get_dict_from_id_list(df):
  for id in df[0]:
    url = 'https://tiki.vn/api/v2/reviews?product_id=' + str(id) + "&sort=score%7Cdesc,id%7Cdesc,stars%7Call&page=1&limit=10&include=comments"
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    list_data.append(data)
  return list_data

In [ ]:
names = ['giay+cao+got', 'sach+giao+khoa', 'truyện+tranh', 'ao+somi', 'do+dung+hoc+tap',
         'mu+bao+hiem', 'hoodie', 'ao+len', 'tat', 'tai+nghe+bluetooth', 'tinh+dầu', 'balo', 'quan+au', 'noi+com+dien', "kho+ga", "laptop"
         , 'tã+quần', 'bột+giặt', 'bàn+học', 'đèn+học', 'lăn+khử+mùi', 'nước+hoa']

In [ ]:
incompleteread = ['tinh+dầu', 'noi+com+dien']

In [ ]:
names = ['tinh+dầu']

In [ ]:
for name in names:
  for i in range(1,4):
    contents = []
    titles = []
    ratings = []
    product_id_file = "/content/drive/My Drive/clone_cmt_tiki/id_product/" + name + str(i) + "_id.txt"
    df = pd.read_csv(product_id_file, header=None)
    list_data_loop = get_dict_from_id_list(df)
    for data in list_data_loop:
      for dict_ in data['data']:
        for key, value in dict_.items():
          if key == 'content':
            contents.append(value)
          if key == 'rating':
            ratings.append(value)
          if key == 'title':
            titles.append(value)
    data_save = pd.DataFrame({'ratings': ratings, 'titles': titles, 'comments': contents})
    data_frame_notna = data_save.dropna()
    path = "/content/drive/My Drive/clone_cmt_tiki/data_csv/" + name + str(i) + ".csv"
    data_frame_notna.to_csv(path, index=False)
    print("Saved ", name)


Saved  tinh+dầu
Saved  tinh+dầu
Saved  tinh+dầu


In [ ]:
#save data as csv file
path = '/content/drive/My Drive/devC/clone_data/data_comment_tiki/' + name + '1' + '.csv'
data_frame_notna.to_csv(path, index=False)